# Multi-Domain Specialized Agent System

This notebook implements a multi-agent system with specialized domains (Mechanical, Electrical, Programming) 
and multiple output agents for generating different types of content:

1. **Domain Experts**:
   - Mechanical Engineering Expert
   - Electrical Engineering Expert  
   - Programming/Software Expert

2. **Output Agents**:
   - Architecture Diagram Generator
   - PowerPoint Generator
   - Word Document Generator
   - PDF Creator
   - Code Generator

The system uses Ollama with Llama 3.2 for reasoning and prompt generation, and external agent functions for specific tasks.

In [146]:
# Basic imports
import os
import json
import uuid
import asyncio
from typing import Dict, List, Any, Optional, Union
from dataclasses import dataclass, asdict
from datetime import datetime
from enum import Enum
from pathlib import Path

# Langchain imports
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import Ollama

# HTTP client for API interactions
try:
    import aiohttp
    AIOHTTP_AVAILABLE = True
    print("✅ aiohttp available for API interactions")
except ImportError:
    AIOHTTP_AVAILABLE = False
    print("⚠️ aiohttp not installed. Install with: pip install aiohttp")

# Langchain setup
try:
    from langchain.llms import Ollama
    LANGCHAIN_AVAILABLE = True
    print("✅ Langchain available for agent coordination")
except ImportError:
    LANGCHAIN_AVAILABLE = False
    print("⚠️ Langchain not installed. Install with: pip install langchain")

print("✅ Basic imports loaded successfully!")

# Create storage directory for outputs if it doesn't exist
DATA_DIR = Path("./data")
DATA_DIR.mkdir(exist_ok=True)

✅ aiohttp available for API interactions
✅ Langchain available for agent coordination
✅ Basic imports loaded successfully!


In [156]:
# Core data structures
@dataclass
class DomainExpertInput:
    """Input for a domain expert"""
    user_query: str
    context: Optional[str] = None
    domain_name: str = ""
    additional_instructions: Optional[str] = None
    
@dataclass
class DomainExpertOutput:
    """Output from a domain expert"""
    domain: str
    analysis: str  
    concerns: List[str]
    recommendations: List[str]
    compatibility_notes: Optional[List[str]] = None
    timestamp: str = datetime.now().isoformat()
    
@dataclass 
class GeneratedPrompt:
    """Container for generated prompts"""
    prompt_type: str  # 'domain' or 'agent'
    agent_name: str
    prompt_content: str
    timestamp: str = datetime.now().isoformat()
    file_path: Optional[str] = None
    
@dataclass
class WorkflowStep:
    """Represents a step in the agent workflow"""
    step_id: str
    agent_type: str
    dependencies: List[str]  # List of step_ids this step depends on
    accumulated_prompt: str = ""  # Combined prompt from all previous steps
    generated_prompt: Optional[GeneratedPrompt] = None
    executed: bool = False
    output: Optional[Any] = None

@dataclass
class AgentExecutionRequest:
    """Request to execute a specific agent"""
    agent_type: str
    user_query: str
    domain_outputs: Dict[str, DomainExpertOutput]
    workflow_context: Dict[str, WorkflowStep] = None  # Added workflow context
    specific_instructions: Optional[str] = None
    
@dataclass
class AgentOutput:
    """Output from a specific agent"""
    agent_type: str
    content: Any
    format: str
    file_path: Optional[str] = None
    execution_time: float = 0.0
    timestamp: str = datetime.now().isoformat()
    
@dataclass
class SystemState:
    """Overall system state"""
    conversation_id: str
    user_query: str
    domain_outputs: Dict[str, DomainExpertOutput]
    agent_outputs: Dict[str, AgentOutput] 
    conversation_history: List[Dict[str, str]]
    generated_prompts: Dict[str, GeneratedPrompt] = None  # Store all prompts
    workflow_steps: Dict[str, WorkflowStep] = None  # Store workflow steps
    last_updated: str = datetime.now().isoformat()
    
    def __post_init__(self):
        """Initialize default values"""
        if self.generated_prompts is None:
            self.generated_prompts = {}
        if self.workflow_steps is None:
            self.workflow_steps = {}
    
    def save_to_json(self, file_path: Optional[str] = None) -> str:
        """Save system state to JSON file"""
        if file_path is None:
            file_path = f"./data/system_state_{self.conversation_id[:8]}.json"
        
        # Convert to dictionary with proper serialization
        state_dict = {
            "conversation_id": self.conversation_id,
            "user_query": self.user_query,
            "domain_outputs": {k: asdict(v) for k, v in self.domain_outputs.items()},
            "agent_outputs": {k: asdict(v) for k, v in self.agent_outputs.items()},
            "conversation_history": self.conversation_history,
            "generated_prompts": {k: asdict(v) for k, v in self.generated_prompts.items()},
            "workflow_steps": {k: asdict(v) for k, v in self.workflow_steps.items()},
            "last_updated": self.last_updated
        }
        
        with open(file_path, 'w') as f:
            json.dump(state_dict, f, indent=2)
        
        return file_path
    
    @classmethod
    def load_from_json(cls, file_path: str) -> 'SystemState':
        """Load system state from JSON file"""
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Convert dictionaries back to dataclasses
        domain_outputs = {k: DomainExpertOutput(**v) for k, v in data['domain_outputs'].items()}
        agent_outputs = {k: AgentOutput(**v) for k, v in data['agent_outputs'].items()}
        generated_prompts = {k: GeneratedPrompt(**v) for k, v in data.get('generated_prompts', {}).items()}
        workflow_steps = {k: WorkflowStep(**v) for k, v in data.get('workflow_steps', {}).items()}
        
        return cls(
            conversation_id=data['conversation_id'],
            user_query=data['user_query'],
            domain_outputs=domain_outputs,
            agent_outputs=agent_outputs,
            conversation_history=data['conversation_history'],
            generated_prompts=generated_prompts,
            workflow_steps=workflow_steps,
            last_updated=data['last_updated']
        )

# Define domain expert types
class DomainType(Enum):
    MECHANICAL = "mechanical"
    ELECTRICAL = "electrical"
    PROGRAMMING = "programming"
    
# Define agent types
class AgentType(Enum):
    DIAGRAM = "diagram"
    PRESENTATION = "presentation"
    DOCUMENT = "document"
    PDF = "pdf"
    CODE = "code"

print("✅ Core data structures defined with workflow support!")

✅ Core data structures defined with workflow support!


In [148]:
# LLM Configuration
class LLMConfig:
    """Configuration for LLM interactions"""
    
    def __init__(
        self,
        base_url: str = "http://127.0.0.1:11434",
        model_name: str = "llama3.2",  # or llama3:8b, llama3:70b based on your local setup
        temperature: float = 0.7,
        timeout: int = 60
    ):
        self.base_url = base_url
        self.model_name = model_name
        self.temperature = temperature
        self.timeout = timeout
        
        
    def get_langchain_llm(self):
        """Get LLM instance for Langchain"""
        if not LANGCHAIN_AVAILABLE:
            print("⚠️ Langchain not available, using mock responses")
            exit(0)
        
            
        try:
            return Ollama(
                model=self.model_name,
                base_url=self.base_url,
                temperature=self.temperature,
                timeout=self.timeout
            )
        except Exception as e:
            print(f"❌ Error creating Ollama LLM: {e}")
            print("🎭 Falling back to mock responses")
            exit(0)
            
        

# Test LLM configuration  
llm_config = LLMConfig()  # Set to False when Ollama is available
print("✅ LLM configuration created!")

✅ LLM configuration created!


In [157]:
# Workflow Manager for Sequential Agent Execution
class WorkflowManager:
    """Manages sequential workflow execution with prompt chaining"""
    
    def __init__(self, conversation_id: str):
        self.conversation_id = conversation_id
        self.workflow_steps = {}
        self.execution_order = []
        
    def create_workflow(self, agent_types: List[str]) -> Dict[str, WorkflowStep]:
        """Create a sequential workflow for the given agent types"""
        self.workflow_steps = {}
        self.execution_order = agent_types.copy()
        
        for i, agent_type in enumerate(agent_types):
            step_id = f"step_{i+1}_{agent_type}"
            dependencies = [f"step_{i}_{agent_types[i-1]}"] if i > 0 else []
            
            self.workflow_steps[step_id] = WorkflowStep(
                step_id=step_id,
                agent_type=agent_type,
                dependencies=dependencies
            )
        
        return self.workflow_steps
    
    def get_accumulated_prompt(self, step_id: str, domain_prompts: Dict[str, str]) -> str:
        """Get accumulated prompt for a specific step"""
        step = self.workflow_steps[step_id]
        accumulated_prompt = ""
        
        # Add domain expert prompts first
        domain_section = "DOMAIN EXPERT ANALYSES:\n\n"
        for domain, prompt in domain_prompts.items():
            domain_section += f"=== {domain.upper()} DOMAIN ANALYSIS ===\n{prompt}\n\n"
        
        accumulated_prompt += domain_section
        
        # Add previous agent prompts
        current_step_index = self.execution_order.index(step.agent_type)
        if current_step_index > 0:
            agent_section = "PREVIOUS AGENT PROMPTS:\n\n"
            for i in range(current_step_index):
                prev_agent_type = self.execution_order[i]
                prev_step_id = f"step_{i+1}_{prev_agent_type}"
                prev_step = self.workflow_steps[prev_step_id]
                
                if prev_step.generated_prompt:
                    agent_section += f"=== {prev_agent_type.upper()} AGENT PROMPT ===\n"
                    agent_section += f"{prev_step.generated_prompt.prompt_content}\n\n"
            
            accumulated_prompt += agent_section
        
        return accumulated_prompt
    
    def update_step_prompt(self, step_id: str, generated_prompt: GeneratedPrompt):
        """Update a workflow step with generated prompt"""
        if step_id in self.workflow_steps:
            self.workflow_steps[step_id].generated_prompt = generated_prompt
            self.workflow_steps[step_id].accumulated_prompt = generated_prompt.prompt_content
    
    def mark_step_executed(self, step_id: str, output: Any):
        """Mark a step as executed with output"""
        if step_id in self.workflow_steps:
            self.workflow_steps[step_id].executed = True
            self.workflow_steps[step_id].output = output
    
    def get_steps_to_redo(self, changed_step_id: str) -> List[str]:
        """Get list of steps that need to be redone after a change"""
        changed_step_index = None
        for i, agent_type in enumerate(self.execution_order):
            step_id = f"step_{i+1}_{agent_type}"
            if step_id == changed_step_id:
                changed_step_index = i
                break
        
        if changed_step_index is None:
            return []
        
        # Return all steps from the changed step onwards
        steps_to_redo = []
        for i in range(changed_step_index, len(self.execution_order)):
            agent_type = self.execution_order[i]
            step_id = f"step_{i+1}_{agent_type}"
            steps_to_redo.append(step_id)
        
        return steps_to_redo
    
    def reset_steps(self, step_ids: List[str]):
        """Reset specified steps to unexecuted state"""
        for step_id in step_ids:
            if step_id in self.workflow_steps:
                self.workflow_steps[step_id].executed = False
                self.workflow_steps[step_id].output = None
    
    def save_prompts_to_files(self, base_path: str = "./data"):
        """Save all generated prompts to individual files"""
        saved_files = {}
        
        for step_id, step in self.workflow_steps.items():
            if step.generated_prompt:
                filename = f"{step.agent_type}_prompt_{self.conversation_id[:8]}.txt"
                file_path = os.path.join(base_path, filename)
                
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(f"# {step.agent_type.upper()} AGENT PROMPT\n")
                    f.write(f"# Generated: {step.generated_prompt.timestamp}\n")
                    f.write(f"# Step ID: {step_id}\n\n")
                    f.write(step.generated_prompt.prompt_content)
                
                step.generated_prompt.file_path = file_path
                saved_files[step.agent_type] = file_path
        
        return saved_files

print("✅ Workflow manager implemented!")

✅ Workflow manager implemented!


In [159]:
# Domain Experts Implementation with Prompt Saving
class DomainExpert:
    """Base class for domain experts"""
    
    def __init__(
        self,
        domain_type: DomainType,
        llm_config: LLMConfig
    ):
        self.domain_type = domain_type
        self.llm = llm_config.get_langchain_llm()
        self.system_prompt = self._get_domain_system_prompt()
        
    def _get_domain_system_prompt(self) -> str:
        """Get domain-specific system prompt"""
        if self.domain_type == DomainType.MECHANICAL:
            return """You are an expert Mechanical Engineer with extensive experience in designing physical systems, 
mechanisms, structures, and manufacturing processes. Think exclusively from a mechanical engineering perspective.

When analyzing problems:
1. Focus on physical principles, materials, structural integrity, and mechanical systems.
2. Consider forces, stresses, thermal effects, vibration, and material properties.
3. Evaluate manufacturability, assembly, maintenance, and mechanical reliability.
4. Identify potential mechanical failure points and physical constraints.
5. Always prioritize safety, durability, and mechanical efficiency.

Provide specific mechanical engineering insights with technical depth. Don't discuss electrical or software aspects 
unless they directly impact the mechanical design. Use precise mechanical engineering terminology.

Your analysis should include:
- Core mechanical principles that apply
- Material recommendations
- Structural considerations
- Thermal and vibration management
- Manufacturing approach
- Mechanical limitations and concerns"""

        elif self.domain_type == DomainType.ELECTRICAL:
            return """You are an expert Electrical Engineer with extensive experience in designing circuits, power systems, 
and electronic components. Think exclusively from an electrical engineering perspective.

When analyzing problems:
1. Focus on electrical principles, circuit design, power distribution, and signal integrity.
2. Consider voltage levels, current requirements, power management, and EMI/EMC concerns.
3. Evaluate electrical components, PCB design, wiring, and electrical safety.
4. Identify potential electrical failure modes and constraints.
5. Always prioritize electrical safety, reliability, and efficiency.

Provide specific electrical engineering insights with technical depth. Don't discuss mechanical or software aspects 
unless they directly impact the electrical design. Use precise electrical engineering terminology.

Your analysis should include:
- Power requirements and distribution
- Circuit design considerations
- Component selection guidelines
- Signal integrity and noise concerns
- Electrical safety measures
- Testing and validation approaches"""

        elif self.domain_type == DomainType.PROGRAMMING:
            return """You are an expert Software Engineer with extensive experience in designing software architectures, 
algorithms, and embedded systems. Think exclusively from a software engineering perspective.

When analyzing problems:
1. Focus on software architecture, data structures, algorithms, and system design.
2. Consider execution efficiency, memory usage, maintainability, and scalability.
3. Evaluate appropriate programming languages, frameworks, and development methodologies.
4. Identify potential software failure modes and technical debt.
5. Always prioritize code quality, maintainability, and performance.

Provide specific software engineering insights with technical depth. Don't discuss mechanical or electrical aspects 
unless they directly impact the software design. Use precise software engineering terminology.

Your analysis should include:
- Software architecture recommendations
- Data structure and algorithm considerations
- Language and framework selection
- Testing strategies
- Error handling approaches
- Performance optimization opportunities"""
        
        else:
            return "You are an engineering expert. Analyze the problem and provide technical insights."

    async def analyze(self, input_data: DomainExpertInput, conversation_id: str) -> tuple[DomainExpertOutput, GeneratedPrompt]:
        """Analyze the input from domain perspective and return both output and generated prompt"""
        
        # Create the prompt for this domain analysis
        context_section = f"ADDITIONAL CONTEXT:\n{input_data.context}" if input_data.context else ""
        instructions_section = f"SPECIFIC INSTRUCTIONS:\n{input_data.additional_instructions}" if input_data.additional_instructions else ""
        
        analysis_prompt = f"""
Analyze this requirement from your {self.domain_type.value} engineering perspective:

USER REQUIREMENT:
{input_data.user_query}

{context_section}

{instructions_section}

Provide your analysis in this format:
1. Core principles and considerations from your domain
2. Key concerns and potential issues
3. Specific recommendations and approaches

Be thorough, technical, and focus exclusively on {self.domain_type.value} engineering aspects.
"""

        # Save the generated prompt
        generated_prompt = GeneratedPrompt(
            prompt_type="domain",
            agent_name=f"{self.domain_type.value}_expert",
            prompt_content=analysis_prompt,
            timestamp=datetime.now().isoformat()
        )
        
        # Save prompt to file
        prompt_filename = f"{self.domain_type.value}_domain_prompt_{conversation_id[:8]}.txt"
        prompt_file_path = f"./data/{prompt_filename}"
        with open(prompt_file_path, 'w', encoding='utf-8') as f:
            f.write(f"# {self.domain_type.value.upper()} DOMAIN EXPERT PROMPT\n")
            f.write(f"# Generated: {generated_prompt.timestamp}\n\n")
            f.write(analysis_prompt)
        
        generated_prompt.file_path = prompt_file_path
        
        # Create the actual LLM prompt template
        prompt_template = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=analysis_prompt)
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt_template)
        
        # Time the execution
        start_time = datetime.now()
        result = await chain.ainvoke({"input": input_data})
        execution_time = (datetime.now() - start_time).total_seconds()
        
        # Process the output
        analysis = result['text'] if isinstance(result, dict) and 'text' in result else str(result)
        
        # Extract key points
        concerns = self._extract_concerns(analysis)
        recommendations = self._extract_recommendations(analysis)
        
        domain_output = DomainExpertOutput(
            domain=self.domain_type.value,
            analysis=analysis,
            concerns=concerns,
            recommendations=recommendations
        )
        
        return domain_output, generated_prompt
    
    def _extract_concerns(self, analysis: str) -> List[str]:
        """Extract key concerns from analysis"""
        # Simple extraction based on key phrases and formatting
        concerns = []
        lines = analysis.split('\n')
        in_concerns_section = False
        
        for line in lines:
            if "concerns" in line.lower() or "issues" in line.lower():
                in_concerns_section = True
                continue
                
            if in_concerns_section and (line.strip() == "" or "recommendations" in line.lower()):
                in_concerns_section = False
                continue
                
            if in_concerns_section and line.strip():
                # Clean up bullet points and numbering
                clean_line = line.strip()
                for prefix in ['-', '•', '*', '○', '➢', '→']:
                    if clean_line.startswith(prefix):
                        clean_line = clean_line[1:].strip()
                        
                # Remove numbering like "1." or "1)"
                if clean_line and clean_line[0].isdigit() and clean_line[1:3] in ['. ', ') ']:
                    clean_line = clean_line[3:].strip()
                    
                if clean_line:
                    concerns.append(clean_line)
        
        # If we couldn't extract structured concerns, do a simpler extraction
        if not concerns:
            concerns = [line.strip() for line in analysis.split('\n') 
                     if "concern" in line.lower() or "issue" in line.lower()]
        
        return concerns[:5]  # Limit to top 5 concerns
    
    def _extract_recommendations(self, analysis: str) -> List[str]:
        """Extract recommendations from analysis"""
        # Similar to concerns extraction
        recommendations = []
        lines = analysis.split('\n')
        in_recommendations_section = False
        
        for line in lines:
            if "recommendation" in line.lower() or "approach" in line.lower() or "suggest" in line.lower():
                in_recommendations_section = True
                continue
                
            if in_recommendations_section and line.strip() == "":
                in_recommendations_section = False
                continue
                
            if in_recommendations_section and line.strip():
                # Clean up bullet points and numbering
                clean_line = line.strip()
                for prefix in ['-', '•', '*', '○', '➢', '→']:
                    if clean_line.startswith(prefix):
                        clean_line = clean_line[1:].strip()
                        
                # Remove numbering like "1." or "1)"
                if clean_line and clean_line[0].isdigit() and clean_line[1:3] in ['. ', ') ']:
                    clean_line = clean_line[3:].strip()
                    
                if clean_line:
                    recommendations.append(clean_line)
        
        # If we couldn't extract structured recommendations, do a simpler extraction
        if not recommendations:
            recommendations = [line.strip() for line in analysis.split('\n') 
                           if "recommend" in line.lower() or "should" in line.lower() or "must" in line.lower()]
        
        return recommendations[:5]  # Limit to top 5 recommendations

# Create domain experts
async def setup_domain_experts(llm_config: LLMConfig) -> Dict[str, DomainExpert]:
    """Setup all domain experts"""
    mechanical_expert = DomainExpert(DomainType.MECHANICAL, llm_config)
    electrical_expert = DomainExpert(DomainType.ELECTRICAL, llm_config) 
    programming_expert = DomainExpert(DomainType.PROGRAMMING, llm_config)
    
    return {
        "mechanical": mechanical_expert,
        "electrical": electrical_expert,
        "programming": programming_expert
    }

print("✅ Domain experts implementation with prompt saving ready!")

✅ Domain experts implementation with prompt saving ready!


In [160]:
# Domain Integration System
class DomainIntegrator:
    """System for integrating analyses across domains"""
    
    def __init__(self, llm_config: LLMConfig):
        self.llm = llm_config.get_langchain_llm()
    
    async def integrate_domain_analyses(
        self, 
        user_query: str,
        domain_outputs: Dict[str, DomainExpertOutput]
    ) -> Dict[str, Any]:
        """Integrate analyses from different domains"""
        
        prompt_template = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are an expert engineering integration specialist who understands 
            mechanical, electrical, and software engineering deeply.
            
            Your task is to analyze separate domain-specific assessments and identify:
            1. Areas of compatibility and alignment between domains
            2. Potential conflicts or contradictions between domains
            3. Integration challenges that must be addressed
            4. Cross-domain risks and dependencies
            5. Unified recommendations that satisfy all domains
            
            Provide a balanced perspective that respects the expertise of each domain
            while finding optimal integration solutions."""),
            
            HumanMessage(content=f"""
            Analyze these domain-specific assessments for the following project:
            
            USER REQUIREMENT:
            {user_query}
            
            MECHANICAL ENGINEERING ASSESSMENT:
            {domain_outputs["mechanical"].analysis if "mechanical" in domain_outputs else "No mechanical assessment provided"}
            
            ELECTRICAL ENGINEERING ASSESSMENT:
            {domain_outputs["electrical"].analysis if "electrical" in domain_outputs else "No electrical assessment provided"}
            
            SOFTWARE ENGINEERING ASSESSMENT:
            {domain_outputs["programming"].analysis if "programming" in domain_outputs else "No software assessment provided"}
            
            Create a comprehensive integration analysis that:
            1. Identifies cross-domain compatibility issues
            2. Highlights contradictions between domain recommendations
            3. Provides a unified set of recommendations that satisfies requirements from all domains
            4. Suggests any necessary trade-offs or compromises
            
            Format your response to clearly address cross-domain integration.
            """)
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt_template)
        
        result = await chain.ainvoke({})
        integration_analysis = result['text'] if isinstance(result, dict) and 'text' in result else str(result)
        
        # Build integration report
        integration_report = {
            "integration_analysis": integration_analysis,
            "cross_domain_issues": self._extract_cross_domain_issues(integration_analysis),
            "unified_recommendations": self._extract_unified_recommendations(integration_analysis),
            "timestamp": datetime.now().isoformat()
        }
        
        return integration_report
    
    def _extract_cross_domain_issues(self, analysis: str) -> List[str]:
        """Extract cross-domain issues from integration analysis"""
        issues = []
        lines = analysis.split('\n')
        in_issues_section = False
        
        for line in lines:
            if any(phrase in line.lower() for phrase in ["cross-domain issue", "conflict", "contradiction", "integration challenge"]):
                in_issues_section = True
                continue
                
            if in_issues_section and (line.strip() == "" or any(phrase in line.lower() for phrase in ["recommendation", "conclusion"])):
                in_issues_section = False
                continue
                
            if in_issues_section and line.strip():
                clean_line = self._clean_bullet_point(line)
                if clean_line:
                    issues.append(clean_line)
        
        # If we couldn't extract structured issues, do a simpler extraction
        if not issues:
            issues = [line.strip() for line in analysis.split('\n')
                      if any(phrase in line.lower() for phrase in ["conflict", "contradiction", "issue"])]
        
        return issues[:7]  # Limit to top 7 issues
    
    def _extract_unified_recommendations(self, analysis: str) -> List[str]:
        """Extract unified recommendations from integration analysis"""
        recommendations = []
        lines = analysis.split('\n')
        in_recommendations_section = False
        
        for line in lines:
            if any(phrase in line.lower() for phrase in ["unified recommendation", "integrated approach"]):
                in_recommendations_section = True
                continue
                
            if in_recommendations_section and line.strip() == "":
                in_recommendations_section = False
                continue
                
            if in_recommendations_section and line.strip():
                clean_line = self._clean_bullet_point(line)
                if clean_line:
                    recommendations.append(clean_line)
        
        # If we couldn't extract structured recommendations, do a simpler extraction
        if not recommendations:
            recommendations = [line.strip() for line in analysis.split('\n')
                           if any(phrase in line.lower() for phrase in ["recommend", "should", "approach"])]
        
        return recommendations[:7]  # Limit to top 7 recommendations
    
    def _clean_bullet_point(self, line: str) -> str:
        """Clean up bullet points and numbering from a line"""
        clean_line = line.strip()
        
        for prefix in ['-', '•', '*', '○', '➢', '→']:
            if clean_line.startswith(prefix):
                clean_line = clean_line[1:].strip()
                
        # Remove numbering like "1." or "1)"
        if clean_line and clean_line[0].isdigit() and len(clean_line) > 2:
            if clean_line[1:3] in ['. ', ') ']:
                clean_line = clean_line[3:].strip()
                
        return clean_line

print("✅ Domain integration system ready!")

✅ Domain integration system ready!


In [162]:
# Agent Prompt Generator with Workflow Integration
class AgentPromptGenerator:
    """Generates specialized prompts for different agents with workflow context"""
    
    def __init__(self, llm_config: LLMConfig):
        self.llm = llm_config.get_langchain_llm()
    
    async def generate_agent_prompt(
        self,
        agent_type: AgentType,
        user_query: str,
        workflow_manager: WorkflowManager,
        step_id: str,
        domain_prompts: Dict[str, str],
        integration_report: Dict[str, Any],
        specific_instructions: Optional[str] = None
    ) -> GeneratedPrompt:
        """Generate specialized prompt for specific agent type with workflow context"""
        
        # Get accumulated context from workflow
        accumulated_prompt = workflow_manager.get_accumulated_prompt(step_id, domain_prompts)
        
        # Get agent-specific system instructions
        system_instruction = self._get_agent_system_instruction(agent_type)
        
        # Integration insights
        integration_insights = f"""
CROSS-DOMAIN INTEGRATION INSIGHTS:
{integration_report['integration_analysis'][:500]}...

Key integration issues: {', '.join(integration_report['cross_domain_issues'][:3])}
Unified recommendations: {', '.join(integration_report['unified_recommendations'][:3])}
"""
        
        # Create the comprehensive prompt
        instructions_section = f"SPECIFIC INSTRUCTIONS FOR THIS AGENT:\n{specific_instructions}" if specific_instructions else ""
        
        agent_prompt_content = f"""
{system_instruction}

ORIGINAL USER REQUEST:
{user_query}

{accumulated_prompt}

{integration_insights}

{instructions_section}

TASK:
Based on all the above context and analyses, create content that addresses the user's original request.
Incorporate insights from all domain experts and previous agent work to create a comprehensive output.
Ensure your output builds upon and complements the work done by previous agents in the workflow.
"""
        
        # Generate enhanced prompt using LLM
        prompt_template = ChatPromptTemplate.from_messages([
            SystemMessage(content=f"""You are a prompt engineering specialist for {agent_type.value} creation.
            Create an optimized prompt for a specialized {agent_type.value} generation agent that incorporates
            all the provided context and produces the best possible {agent_type.value}."""),
            
            HumanMessage(content=agent_prompt_content)
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt_template)
        result = await chain.ainvoke({})
        
        enhanced_prompt = result['text'] if isinstance(result, dict) and 'text' in result else str(result)
        
        # Create the generated prompt object
        generated_prompt = GeneratedPrompt(
            prompt_type="agent",
            agent_name=agent_type.value,
            prompt_content=enhanced_prompt,
            timestamp=datetime.now().isoformat()
        )
        
        return generated_prompt
    
    def _get_agent_system_instruction(self, agent_type: AgentType) -> str:
        """Get system instruction specific to agent type"""
        if agent_type == AgentType.DIAGRAM:
            return """AGENT TYPE: ARCHITECTURE DIAGRAM GENERATOR

You specialize in creating technical and architectural diagrams. Your output should be:
- Clear visual representations of systems and relationships
- Appropriate diagram types (UML, flowcharts, system architecture, etc.)
- Proper notation and symbols
- Clear component relationships and data flows
- Scalable and maintainable design representations

Focus on translating complex technical requirements into visual diagrams that communicate 
the system architecture, component interactions, and design decisions effectively."""
            
        elif agent_type == AgentType.PRESENTATION:
            return """AGENT TYPE: PRESENTATION GENERATOR

You specialize in creating PowerPoint presentations. Your output should include:
- Well-structured slide layouts with clear narrative flow
- Executive summary and key takeaways
- Technical details appropriate for the audience
- Visual elements and data visualizations
- Compelling storytelling that explains the solution
- Action items and next steps

Focus on creating presentations that effectively communicate technical solutions 
to stakeholders while maintaining engagement and clarity."""
            
        elif agent_type == AgentType.DOCUMENT:
            return """AGENT TYPE: TECHNICAL DOCUMENT GENERATOR

You specialize in creating comprehensive technical documentation. Your output should include:
- Structured document layout with proper sections
- Detailed technical specifications
- Implementation guidelines and best practices
- Risk assessments and mitigation strategies
- Requirements traceability
- Professional formatting and readability

Focus on creating documents that serve as complete references for technical 
implementation and decision-making."""
            
        elif agent_type == AgentType.PDF:
            return """AGENT TYPE: PDF REPORT GENERATOR

You specialize in creating professional PDF reports. Your output should include:
- Executive summary for decision-makers
- Detailed technical analysis and findings
- Data visualizations and charts
- Recommendations and action plans
- Appendices with supporting information
- Professional layout and formatting

Focus on creating comprehensive reports that combine technical depth 
with executive-level clarity and visual appeal."""
            
        elif agent_type == AgentType.CODE:
            return """AGENT TYPE: CODE GENERATOR

You specialize in generating implementation code. Your output should include:
- Clean, well-documented code following best practices
- Appropriate architecture patterns and design principles
- Error handling and validation
- Unit tests and integration examples
- Configuration files and deployment scripts
- Documentation and README files

Focus on creating production-ready code that implements the technical 
requirements while being maintainable and scalable."""
            
        else:
            return f"""AGENT TYPE: {agent_type.value.upper()} GENERATOR

You specialize in creating {agent_type.value} content. Focus on delivering high-quality 
output that meets the user's requirements while incorporating insights from domain experts."""

print("✅ Agent prompt generator with workflow integration ready!")

✅ Agent prompt generator with workflow integration ready!


In [163]:
# Mock External Agent Functions
# In a real implementation, these would call your external agent functions

async def call_diagram_agent(prompt: str) -> AgentOutput:
    """Call external diagram generation agent"""
    print(f"🔄 Calling external diagram agent with prompt length: {len(prompt)}")
    await asyncio.sleep(1)  # Simulate processing time
    
    return AgentOutput(
        agent_type=AgentType.DIAGRAM.value,
        content="[Generated architecture diagram with component relationships]",
        format="png",
        file_path="./data/generated_diagram.png",
        execution_time=1.0
    )

async def call_presentation_agent(prompt: str) -> AgentOutput:
    """Call external PowerPoint generation agent"""
    print(f"🔄 Calling external presentation agent with prompt length: {len(prompt)}")
    await asyncio.sleep(1.5)  # Simulate processing time
    
    return AgentOutput(
        agent_type=AgentType.PRESENTATION.value,
        content="[Generated PowerPoint presentation with 12 slides]",
        format="pptx",
        file_path="./data/generated_presentation.pptx",
        execution_time=1.5
    )

async def call_document_agent(prompt: str) -> AgentOutput:
    """Call external document generation agent"""
    print(f"🔄 Calling external document agent with prompt length: {len(prompt)}")
    await asyncio.sleep(1.2)  # Simulate processing time
    
    return AgentOutput(
        agent_type=AgentType.DOCUMENT.value,
        content="[Generated Word document with technical specifications]",
        format="docx",
        file_path="./data/generated_document.docx",
        execution_time=1.2
    )

async def call_pdf_agent(prompt: str) -> AgentOutput:
    """Call external PDF generation agent"""
    print(f"🔄 Calling external PDF agent with prompt length: {len(prompt)}")
    await asyncio.sleep(1.3)  # Simulate processing time
    
    return AgentOutput(
        agent_type=AgentType.PDF.value,
        content="[Generated PDF report with data visualizations]",
        format="pdf",
        file_path="./data/generated_report.pdf",
        execution_time=1.3
    )

async def call_code_agent(prompt: str) -> AgentOutput:
    """Call external code generation agent"""
    print(f"🔄 Calling external code agent with prompt length: {len(prompt)}")
    await asyncio.sleep(1.4)  # Simulate processing time
    
    return AgentOutput(
        agent_type=AgentType.CODE.value,
        content="[Generated code repository with implementation]",
        format="zip",
        file_path="./data/generated_code.zip",
        execution_time=1.4
    )

# Agent execution function map
AGENT_FUNCTION_MAP = {
    AgentType.DIAGRAM.value: call_diagram_agent,
    AgentType.PRESENTATION.value: call_presentation_agent,
    AgentType.DOCUMENT.value: call_document_agent,
    AgentType.PDF.value: call_pdf_agent,
    AgentType.CODE.value: call_code_agent
}

print("✅ Agent function mapping ready!")

✅ Agent function mapping ready!


In [164]:
# Main System Orchestrator with Sequential Workflow
class MultiDomainSystem:
    """Main system for orchestrating multi-domain workflows with sequential agent execution"""
    
    def __init__(self, use_mock: bool = True):
        # Setup configurations
        self.llm_config = LLMConfig(timeout=1500)
        self.conversation_id = str(uuid.uuid4())
        self.conversation_memory = ConversationBufferMemory(return_messages=True)
        self.current_state = None
        self.workflow_manager = WorkflowManager(self.conversation_id)
        
        # Initialize components (will be set up asynchronously)
        self.domain_experts = None
        self.domain_integrator = DomainIntegrator(self.llm_config)
        self.prompt_generator = AgentPromptGenerator(self.llm_config)
    
    async def setup(self):
        """Initialize the system components"""
        self.domain_experts = await setup_domain_experts(self.llm_config)
        print("✅ Multi-domain system initialized and ready!")
    
    async def process_user_query(self, user_query: str, agent_workflow: List[str] = None) -> SystemState:
        """Process a new user query through the entire system with optional workflow"""
        print(f"🔄 Processing user query: {user_query[:100]}{'...' if len(user_query) > 100 else ''}")
        
        # Set default workflow if none provided
        if agent_workflow is None:
            agent_workflow = ["diagram", "presentation", "document", "pdf", "code"]
        
        # Create workflow
        workflow_steps = self.workflow_manager.create_workflow(agent_workflow)
        
        # Add to conversation history
        self.conversation_memory.chat_memory.add_user_message(user_query)
        
        # Step 1: Analyze with domain experts and save their prompts
        domain_outputs = {}
        domain_prompts = {}
        generated_prompts = {}
        
        for domain_name, expert in self.domain_experts.items():
            print(f"🔄 Analyzing with {domain_name} expert...")
            input_data = DomainExpertInput(
                user_query=user_query,
                domain_name=domain_name
            )
            
            # Get both output and generated prompt
            domain_output, generated_prompt = await expert.analyze(input_data, self.conversation_id)
            domain_outputs[domain_name] = domain_output
            domain_prompts[domain_name] = generated_prompt.prompt_content
            generated_prompts[f"{domain_name}_domain"] = generated_prompt
            
            print(f"✅ {domain_name.capitalize()} analysis complete - prompt saved")
        
        # Step 2: Integrate domain analyses
        print("🔄 Integrating domain analyses...")
        integration_report = await self.domain_integrator.integrate_domain_analyses(
            user_query, domain_outputs
        )
        print("✅ Domain integration complete")
        
        # Step 3: Save domain outputs and integration to JSON
        output_files = {}
        for domain, output in domain_outputs.items():
            file_path = f"./data/{domain}_analysis_{self.conversation_id[:8]}.json"
            with open(file_path, 'w') as f:
                json.dump(asdict(output), f, indent=2)
            output_files[domain] = file_path
        
        integration_file = f"./data/integration_{self.conversation_id[:8]}.json"
        with open(integration_file, 'w') as f:
            json.dump(integration_report, f, indent=2)
        output_files["integration"] = integration_file
        
        # Create and save system state
        conversation_history = [{
            "role": msg.type,
            "content": msg.content
        } for msg in self.conversation_memory.chat_memory.messages]
        
        self.current_state = SystemState(
            conversation_id=self.conversation_id,
            user_query=user_query,
            domain_outputs=domain_outputs,
            agent_outputs={},
            conversation_history=conversation_history,
            generated_prompts=generated_prompts,
            workflow_steps=workflow_steps,
            last_updated=datetime.now().isoformat()
        )
        
        # Store domain prompts and integration report for workflow
        self.domain_prompts = domain_prompts
        self.integration_report = integration_report
        
        # Save system state
        state_file = self.current_state.save_to_json()
        print(f"✅ System state saved to {state_file}")
        print(f"✅ Workflow created with {len(agent_workflow)} agents: {', '.join(agent_workflow)}")
        
        # Add summary to conversation
        summary = f"I've analyzed your request across mechanical, electrical, and programming domains and created a workflow for {len(agent_workflow)} agents."
        self.conversation_memory.chat_memory.add_ai_message(summary)
        
        return self.current_state
    
    async def execute_workflow_step(self, step_id: str, specific_instructions: Optional[str] = None) -> AgentOutput:
        """Execute a specific workflow step"""
        if self.current_state is None:
            raise ValueError("No active query to process. Please submit a user query first.")
        
        if step_id not in self.workflow_manager.workflow_steps:
            raise ValueError(f"Unknown step ID: {step_id}")
        
        step = self.workflow_manager.workflow_steps[step_id]
        agent_type = step.agent_type
        
        print(f"🚀 Executing workflow step: {step_id} ({agent_type})")
        
        # Generate agent-specific prompt using workflow context
        agent_enum_type = next(at for at in AgentType if at.value == agent_type)
        generated_prompt = await self.prompt_generator.generate_agent_prompt(
            agent_type=agent_enum_type,
            user_query=self.current_state.user_query,
            workflow_manager=self.workflow_manager,
            step_id=step_id,
            domain_prompts=self.domain_prompts,
            integration_report=self.integration_report,
            specific_instructions=specific_instructions
        )
        
        # Save the generated prompt to workflow and system state
        self.workflow_manager.update_step_prompt(step_id, generated_prompt)
        self.current_state.generated_prompts[f"{agent_type}_agent"] = generated_prompt
        
        # Save prompt to file
        prompt_filename = f"{agent_type}_prompt_{self.conversation_id[:8]}.txt"
        prompt_file_path = f"./data/{prompt_filename}"
        with open(prompt_file_path, 'w', encoding='utf-8') as f:
            f.write(f"# {agent_type.upper()} AGENT PROMPT\n")
            f.write(f"# Generated: {generated_prompt.timestamp}\n")
            f.write(f"# Step ID: {step_id}\n\n")
            f.write(generated_prompt.prompt_content)
        
        generated_prompt.file_path = prompt_file_path
        
        # Call the appropriate agent function
        agent_function = AGENT_FUNCTION_MAP.get(agent_type)
        
        if agent_function:
            agent_output = await agent_function(generated_prompt.prompt_content)
            
            # Update workflow step and system state
            self.workflow_manager.mark_step_executed(step_id, agent_output)
            self.current_state.agent_outputs[agent_type] = agent_output
            self.current_state.workflow_steps = self.workflow_manager.workflow_steps
            self.current_state.save_to_json()
            
            # Add to conversation
            self.conversation_memory.chat_memory.add_ai_message(
                f"I've completed the {agent_type} step in the workflow."
            )
            
            print(f"✅ {agent_type.capitalize()} step complete - prompt and output saved")
            return agent_output
        else:
            raise ValueError(f"No implementation found for agent type: {agent_type}")
    
    async def execute_full_workflow(self) -> Dict[str, AgentOutput]:
        """Execute all steps in the workflow sequentially"""
        if self.current_state is None:
            raise ValueError("No active query to process. Please submit a user query first.")
        
        print("🚀 Executing full workflow...")
        all_outputs = {}
        
        for i, agent_type in enumerate(self.workflow_manager.execution_order):
            step_id = f"step_{i+1}_{agent_type}"
            
            if not self.workflow_manager.workflow_steps[step_id].executed:
                output = await self.execute_workflow_step(step_id)
                all_outputs[agent_type] = output
            else:
                print(f"⏭️ Skipping already executed step: {step_id}")
                all_outputs[agent_type] = self.workflow_manager.workflow_steps[step_id].output
        
        print("✅ Full workflow execution complete!")
        return all_outputs
    
    async def modify_step_prompt(
        self,
        step_id: str,
        user_feedback: str
    ) -> Dict[str, AgentOutput]:
        """Modify a step's prompt and re-execute it and all subsequent steps"""
        if self.current_state is None:
            raise ValueError("No active query to process. Please submit a user query first.")
        
        if step_id not in self.workflow_manager.workflow_steps:
            raise ValueError(f"Unknown step ID: {step_id}")
        
        print(f"🔄 Modifying step {step_id} based on feedback...")
        
        # Get steps that need to be redone
        steps_to_redo = self.workflow_manager.get_steps_to_redo(step_id)
        print(f"📝 Steps to redo: {', '.join(steps_to_redo)}")
        
        # Reset the steps
        self.workflow_manager.reset_steps(steps_to_redo)
        
        # Add feedback to conversation history
        self.conversation_memory.chat_memory.add_user_message(
            f"Feedback for {step_id}: {user_feedback}"
        )
        
        # Re-execute the modified step and all subsequent steps
        all_outputs = {}
        for step_id_to_redo in steps_to_redo:
            step = self.workflow_manager.workflow_steps[step_id_to_redo]
            
            # Use feedback as specific instructions for the first modified step
            specific_instructions = user_feedback if step_id_to_redo == step_id else None
            
            output = await self.execute_workflow_step(step_id_to_redo, specific_instructions)
            all_outputs[step.agent_type] = output
        
        print("✅ Workflow modification complete!")
        return all_outputs
    
    def get_workflow_status(self) -> Dict[str, Any]:
        """Get current workflow status"""
        if not self.workflow_manager.workflow_steps:
            return {"status": "No workflow created"}
        
        status = {
            "total_steps": len(self.workflow_manager.workflow_steps),
            "completed_steps": sum(1 for step in self.workflow_manager.workflow_steps.values() if step.executed),
            "execution_order": self.workflow_manager.execution_order,
            "steps": {}
        }
        
        for step_id, step in self.workflow_manager.workflow_steps.items():
            status["steps"][step_id] = {
                "agent_type": step.agent_type,
                "executed": step.executed,
                "has_prompt": step.generated_prompt is not None,
                "has_output": step.output is not None
            }
        
        return status
    
    def get_all_generated_prompts(self) -> Dict[str, str]:
        """Get all generated prompts with their file paths"""
        if not self.current_state:
            return {}
        
        prompts_info = {}
        for prompt_key, prompt_obj in self.current_state.generated_prompts.items():
            prompts_info[prompt_key] = {
                "type": prompt_obj.prompt_type,
                "agent": prompt_obj.agent_name,
                "file_path": prompt_obj.file_path,
                "timestamp": prompt_obj.timestamp,
                "content_preview": prompt_obj.prompt_content[:200] + "..." if len(prompt_obj.prompt_content) > 200 else prompt_obj.prompt_content
            }
        
        return prompts_info
    
    def get_conversation_history(self) -> List[Dict[str, str]]:
        """Get the current conversation history"""
        return [{
            "role": msg.type,
            "content": msg.content
        } for msg in self.conversation_memory.chat_memory.messages]

print("✅ Multi-domain system orchestrator with sequential workflow ready!")

✅ Multi-domain system orchestrator with sequential workflow ready!


In [165]:
# Setup and test the sequential workflow system
async def setup_and_test_workflow():
    """Setup and test the multi-domain system with sequential workflow"""
    print("🚀 Setting up multi-domain system with sequential workflow...")
    
    # Create and initialize system
    system = MultiDomainSystem()
    await system.setup()
    
    # Process a test query with custom workflow
    test_query = """
    We need to design an autonomous delivery robot for indoor environments like hospitals and offices.
    The robot should be able to navigate corridors, avoid obstacles, carry payloads of up to 5kg,
    and operate for at least 8 hours on a single charge. It should have a secure compartment
    for delivering items and a touchscreen interface for user interaction. The robot needs to
    integrate with building systems like elevators and automatic doors.
    """
    
    print("\n🧪 TESTING WITH SAMPLE QUERY:")
    print(test_query)
    
    # Process query with custom workflow order
    custom_workflow = ["diagram", "code", "presentation"]  # Custom order
    state = await system.process_user_query(test_query, custom_workflow)
    
    print("\n📊 WORKFLOW STATUS AFTER SETUP:")
    status = system.get_workflow_status()
    print(f"Total steps: {status['total_steps']}")
    print(f"Execution order: {status['execution_order']}")
    
    print("\n📝 DOMAIN PROMPTS GENERATED:")
    for prompt_key, prompt_info in system.get_all_generated_prompts().items():
        if prompt_info["type"] == "domain":
            print(f"- {prompt_key}: {prompt_info['file_path']}")
    
    # Execute first step (diagram)
    print("\n🧪 TESTING FIRST WORKFLOW STEP (DIAGRAM):")
    step_1_id = "step_1_diagram"
    diagram_output = await system.execute_workflow_step(step_1_id)
    print(f"Output: {diagram_output.content}")
    
    # Execute second step (code) - this should include diagram prompt in its context
    print("\n🧪 TESTING SECOND WORKFLOW STEP (CODE):")
    step_2_id = "step_2_code"
    code_output = await system.execute_workflow_step(step_2_id)
    print(f"Output: {code_output.content}")
    
    # Test feedback and modification
    print("\n🧪 TESTING WORKFLOW MODIFICATION:")
    feedback = "The code should focus more on the safety systems and emergency protocols."
    modified_outputs = await system.modify_step_prompt(step_2_id, feedback)
    
    print(f"Modified outputs: {list(modified_outputs.keys())}")
    
    # Show all generated prompts
    print("\n📝 ALL GENERATED PROMPTS:")
    all_prompts = system.get_all_generated_prompts()
    for prompt_key, prompt_info in all_prompts.items():
        print(f"- {prompt_key} ({prompt_info['type']}): {prompt_info['file_path']}")
    
    # Show final workflow status
    print("\n📊 FINAL WORKFLOW STATUS:")
    final_status = system.get_workflow_status()
    print(f"Completed: {final_status['completed_steps']}/{final_status['total_steps']} steps")
    
    return system

# Run the test
test_system = await setup_and_test_workflow()

🚀 Setting up multi-domain system with sequential workflow...
✅ Multi-domain system initialized and ready!

🧪 TESTING WITH SAMPLE QUERY:

    We need to design an autonomous delivery robot for indoor environments like hospitals and offices.
    The robot should be able to navigate corridors, avoid obstacles, carry payloads of up to 5kg,
    and operate for at least 8 hours on a single charge. It should have a secure compartment
    for delivering items and a touchscreen interface for user interaction. The robot needs to
    integrate with building systems like elevators and automatic doors.
    
🔄 Processing user query: 
    We need to design an autonomous delivery robot for indoor environments like hospitals and offic...
🔄 Analyzing with mechanical expert...
✅ Mechanical analysis complete - prompt saved
🔄 Analyzing with electrical expert...
✅ Mechanical analysis complete - prompt saved
🔄 Analyzing with electrical expert...
✅ Electrical analysis complete - prompt saved
🔄 Analyzing with 

CancelledError: 

In [ ]:
# Advanced workflow demonstration
async def demonstrate_advanced_workflow():
    """Demonstrate advanced workflow features with sequential prompt chaining"""
    print("🎯 DEMONSTRATING ADVANCED SEQUENTIAL WORKFLOW")
    print("=" * 70)
    
    # Create fresh system
    system = MultiDomainSystem(use_mock=True)
    await system.setup()
    
    # Complex manufacturing query
    complex_query = """
    Our manufacturing company needs a robotic arm system for automating the assembly of electronic devices.
    The system should pick components from bins, place them on PCBs with high precision (±0.1mm),
    and handle soldering operations. We need to control multiple robotic arms in coordination,
    monitor the quality of assembly in real-time with computer vision, and integrate with our
    existing MES (Manufacturing Execution System). The system should be reconfigurable for different
    products with minimal downtime. Safety features are critical since operators will work alongside
    the robots. We plan to deploy this system in 5 production lines over the next year.
    """
    
    print("\n📋 USER QUERY:")
    print(complex_query)
    
    # Create workflow with all agents in specific order
    full_workflow = ["diagram", "code", "document", "presentation", "pdf"]
    
    # Process the query through domain experts
    print("\n🧠 PROCESSING WITH DOMAIN EXPERTS AND CREATING WORKFLOW...")
    state = await system.process_user_query(complex_query, full_workflow)
    
    print("\n📊 DOMAIN ANALYSIS RESULTS:")
    for domain, output in state.domain_outputs.items():
        print(f"\n▶️ {domain.upper()} DOMAIN:")
        print(f"  • Analysis length: {len(output.analysis)} characters")
        print(f"  • Key concerns: {output.concerns[:2]}")
        print(f"  • Recommendations: {output.recommendations[:2]}")
    
    print("\n📝 DOMAIN PROMPTS SAVED:")
    domain_prompts = {k: v for k, v in system.get_all_generated_prompts().items() if "domain" in k}
    for prompt_key, prompt_info in domain_prompts.items():
        print(f"  • {prompt_key}: {prompt_info['file_path']}")
    
    # Execute workflow step by step to show prompt chaining
    print("\n? EXECUTING WORKFLOW WITH PROMPT CHAINING:")
    
    # Step 1: Diagram (uses only domain prompts)
    print("\n▶️ STEP 1: DIAGRAM GENERATION")
    step_1 = await system.execute_workflow_step("step_1_diagram")
    print(f"  • Generated diagram: {step_1.content[:100]}...")
    
    # Step 2: Code (uses domain prompts + diagram prompt)
    print("\n▶️ STEP 2: CODE GENERATION (includes diagram context)")
    step_2 = await system.execute_workflow_step("step_2_code")
    print(f"  • Generated code: {step_2.content[:100]}...")
    
    # Step 3: Document (uses domain + diagram + code prompts)
    print("\n▶️ STEP 3: DOCUMENT GENERATION (includes diagram + code context)")
    step_3 = await system.execute_workflow_step("step_3_document")
    print(f"  • Generated document: {step_3.content[:100]}...")
    
    # Show prompt chaining in action
    print("\n🔍 DEMONSTRATING PROMPT CHAINING:")
    all_prompts = system.get_all_generated_prompts()
    agent_prompts = {k: v for k, v in all_prompts.items() if "agent" in k}
    
    for i, (prompt_key, prompt_info) in enumerate(agent_prompts.items(), 1):
        print(f"\n{i}. {prompt_key.upper()}:")
        print(f"   File: {prompt_info['file_path']}")
        print(f"   Preview: {prompt_info['content_preview'][:150]}...")
        
        # Show how each prompt builds on previous ones
        if i > 1:
            print(f"   📌 This prompt includes context from {i-1} previous step(s)")
    
    # Demonstrate modification and cascade effect
    print("\n🔄 DEMONSTRATING MODIFICATION CASCADE:")
    feedback = """
    The code architecture should use a microservices approach instead of monolithic design.
    Each robotic arm should be a separate service that can be scaled independently.
    Include Docker containerization and Kubernetes orchestration.
    """
    
    print("📝 Applying feedback to code step...")
    modified_outputs = await system.modify_step_prompt("step_2_code", feedback)
    
    print(f"🔄 Modified and re-executed steps: {list(modified_outputs.keys())}")
    print("📌 Note: All steps after 'code' were automatically re-executed with the new context")
    
    # Execute remaining steps
    print("\n▶️ COMPLETING REMAINING WORKFLOW STEPS:")
    remaining_outputs = await system.execute_full_workflow()
    
    print(f"✅ All steps completed: {list(remaining_outputs.keys())}")
    
    # Show final prompt files
    print("\n? FINAL GENERATED PROMPT FILES:")
    final_prompts = system.get_all_generated_prompts()
    for prompt_key, prompt_info in final_prompts.items():
        print(f"  • {prompt_key}: {prompt_info['file_path']}")
    
    # Show workflow status
    print("\n📊 FINAL WORKFLOW STATUS:")
    final_status = system.get_workflow_status()
    print(f"  • Total steps: {final_status['total_steps']}")
    print(f"  • Completed steps: {final_status['completed_steps']}")
    print(f"  • Success rate: {final_status['completed_steps']/final_status['total_steps']*100:.1f}%")
    
    print("\n✅ ADVANCED WORKFLOW DEMONSTRATION COMPLETE")
    print("🔑 KEY FEATURES DEMONSTRATED:")
    print("   • Sequential prompt chaining (each agent builds on previous)")
    print("   • Automatic prompt saving for all domain and agent interactions")
    print("   • Modification cascade (changing one step updates all subsequent steps)")
    print("   • Comprehensive workflow tracking and status reporting")

# Run the advanced demonstration
await demonstrate_advanced_workflow()

In [166]:
# Usage Examples and System Guide
async def show_usage_examples():
    """Show how to use the enhanced system with all its features"""
    print("📚 SYSTEM USAGE GUIDE")
    print("=" * 50)
    
    print("""
🔧 BASIC USAGE:
    
    # 1. Create and setup system
    system = MultiDomainSystem()
    await system.setup()
    
    # 2. Process query with default workflow
    state = await system.process_user_query("Your engineering question here")
    
    # 3. Execute full workflow
    outputs = await system.execute_full_workflow()

🔧 CUSTOM WORKFLOW:
    
    # Create custom agent order
    custom_workflow = ["code", "diagram", "presentation"]  # Code first, then diagram, then presentation
    state = await system.process_user_query("Your question", custom_workflow)

🔧 STEP-BY-STEP EXECUTION:
    
    # Execute individual steps
    diagram_output = await system.execute_workflow_step("step_1_diagram")
    code_output = await system.execute_workflow_step("step_2_code")
    
🔧 MODIFY AND RE-EXECUTE:
    
    # Modify a step and cascade changes
    feedback = "Make the code more modular with better error handling"
    modified_outputs = await system.modify_step_prompt("step_2_code", feedback)

🔧 CHECK STATUS AND PROMPTS:
    
    # Get workflow status
    status = system.get_workflow_status()
    
    # Get all generated prompts
    prompts = system.get_all_generated_prompts()
    
    # Get conversation history
    history = system.get_conversation_history()
""")
    
    print("\n📁 FILE ORGANIZATION:")
    print("""
    All outputs are saved in ./data/ directory:
    
    📂 data/
    ├── 📄 mechanical_domain_prompt_[id].txt      # Domain expert prompts
    ├── 📄 electrical_domain_prompt_[id].txt
    ├── 📄 programming_domain_prompt_[id].txt
    ├── 📄 diagram_prompt_[id].txt               # Agent prompts
    ├── 📄 code_prompt_[id].txt
    ├── 📄 presentation_prompt_[id].txt
    ├── 📄 document_prompt_[id].txt
    ├── 📄 pdf_prompt_[id].txt
    ├── 📄 system_state_[id].json               # Complete system state
    ├── 📄 integration_[id].json                # Domain integration report
    └── 📄 [domain]_analysis_[id].json          # Individual domain analyses
    """)
    
    print("\n🔑 KEY FEATURES:")
    print("""
    ✅ Sequential Workflow: Each agent's prompt includes all previous agents' prompts
    ✅ Prompt Saving: Every prompt (domain + agent) is automatically saved
    ✅ Modification Cascade: Changing one step automatically updates all subsequent steps
    ✅ Flexible Ordering: Define custom agent execution order
    ✅ State Management: Complete system state is saved and can be restored
    ✅ Progress Tracking: Monitor workflow progress and status
    """)
    
    print("\n⚡ WORKFLOW BENEFITS:")
    print("""
    🔗 Prompt Chaining: Each agent gets context from all previous work
    📝 Complete Traceability: All prompts are saved for audit and debugging  
    🔄 Easy Iteration: Modify any step and automatically update downstream work
    🎯 Targeted Output: Each agent builds upon previous work for better results
    📊 Full Visibility: Track progress and see all generated content
    """)

# Show the usage guide
await show_usage_examples()

📚 SYSTEM USAGE GUIDE

🔧 BASIC USAGE:
    
    # 1. Create and setup system
    system = MultiDomainSystem()
    await system.setup()
    
    # 2. Process query with default workflow
    state = await system.process_user_query("Your engineering question here")
    
    # 3. Execute full workflow
    outputs = await system.execute_full_workflow()

🔧 CUSTOM WORKFLOW:
    
    # Create custom agent order
    custom_workflow = ["code", "diagram", "presentation"]  # Code first, then diagram, then presentation
    state = await system.process_user_query("Your question", custom_workflow)

🔧 STEP-BY-STEP EXECUTION:
    
    # Execute individual steps
    diagram_output = await system.execute_workflow_step("step_1_diagram")
    code_output = await system.execute_workflow_step("step_2_code")
    
🔧 MODIFY AND RE-EXECUTE:
    
    # Modify a step and cascade changes
    feedback = "Make the code more modular with better error handling"
    modified_outputs = await system.modify_step_prompt("step_

## ✅ System Implementation Complete

### 🔄 **Sequential Workflow System**

The Multi-Domain System now includes a **sequential workflow** where each agent's prompt is built upon all previous agents' prompts, creating a chain of context that improves the quality of each subsequent output.

### 📝 **Prompt Saving System**

**Every prompt is automatically saved**:
- **Domain Expert Prompts**: Saved for mechanical, electrical, and programming analyses
- **Agent Prompts**: Saved for each agent (diagram, code, presentation, document, PDF)
- **File Organization**: All prompts stored in `./data/` with timestamps and unique IDs

### 🔗 **Workflow Chaining Logic**

1. **Domain Analysis**: Each domain expert analyzes the user query and saves its prompt
2. **Agent Execution**: Each agent receives:
   - All domain expert prompts and analyses
   - All previous agent prompts in the workflow
   - Integration report from domain synthesis
   - User's original query
3. **Sequential Building**: Agent N+1 gets context from Agents 1 through N

### 🔄 **Modification Cascade**

When you modify any agent's prompt:
- That agent and **all subsequent agents** are automatically re-executed
- The modified context propagates through the entire remaining workflow
- Previous agents remain unchanged

### 💾 **Complete Traceability**

- All prompts saved to individual text files
- System state saved as JSON for full reproducibility
- Conversation history maintained
- Workflow progress tracking

### 🎯 **Key Benefits Achieved**

✅ **Prompt Chaining**: Each agent builds on all previous work  
✅ **Full Prompt Saving**: Every prompt (domain + agent) preserved  
✅ **Modification Cascade**: Changes automatically propagate  
✅ **Flexible Ordering**: Custom agent execution sequences  
✅ **Complete Audit Trail**: All decisions and prompts tracked  
✅ **State Management**: Full system state preservation  

The system now works exactly as requested: **text-only input chaining** where each agent receives the **combined prompts** of all previous agents, enabling cumulative knowledge building without requiring agents to process the actual outputs of other agents.